In [2]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from bs4 import BeautifulSoup

# Initialize the WebDriver
driver = webdriver.Chrome()  # Use the appropriate driver for your browser
main_page_url = "https://www.chelsea.co.nz/recipes"  # Replace with the correct URL
driver.get(main_page_url)

# Wait for the page to load fully
time.sleep(10)

# Wait for an element to be present on the page
def wait_for_element(selector, timeout=30):
    WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))

def load_more_reviews():
    """Click 'Load More' until the button is no longer visible or clickable."""
    while True:
        try:
            # Wait for the 'Load More' button to appear
            load_more_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.cta_button.load-more'))
            )
            
            if load_more_button.is_displayed():
                # Ensure the button is visible and click it
                driver.execute_script("arguments[0].scrollIntoView(true);", load_more_button)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", load_more_button)
                print("Clicked 'Load More' button.")
                time.sleep(10)  # Wait for the next set of recipes to load
            else:
                print("No more 'Load More' button visible.")
                break
        except Exception as e:
            print(f"Error or 'Load More' button no longer available: {e}")
            break

def extract_recipe_details(recipe_url):
    """Extract recipe details from the given URL."""
    driver.get(recipe_url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract pretitle (category)
    pretitle = soup.find('div', class_='pretitle')
    pretitle_text = pretitle.text.strip() if pretitle else 'No Category'

    # Extract title
    title = soup.find('h1', class_='pwr-heading-style--h2 heading')
    title_text = title.text.strip() if title else 'No Title'

    # Extract difficulty
    difficulty = soup.find('div', class_='difficulty')
    difficulty_text = (
        difficulty.find('div', class_='solution').text.strip()
        if difficulty and difficulty.find('div', 'solution')
        else 'No Difficulty Information'
    )

    # Extract recipe steps (method items)
    steps = []
    method_items = soup.find_all('div', class_='method_item')
    for item in method_items:
        description = item.find('div', class_='description')
        if description:
            steps.append(description.text.strip())

    # Join steps into a single text block
    steps_text = ' '.join(steps)

    # Return all collected details
    return {
        'pretitle': pretitle_text,
        'title': title_text,
        'difficulty': difficulty_text,
        'steps': steps_text,
    }

def scrape_recipe_links():
    """Scrape all recipe links from the current page."""
    recipe_links = []
    wait_for_element(".cst-content-grp .item-title h5 a")
    recipe_elements = driver.find_elements(By.CSS_SELECTOR, ".cst-content-grp .item-title h5 a")
    for element in recipe_elements:
        recipe_links.append(element.get_attribute('href'))
    return recipe_links

def scrape_recipes():
    """Scrape recipes, save data periodically, and handle errors."""
    all_recipes = []
    batch_size = 50  # Save after every 50 recipes
    batch_count = 0

    try:
        print("Loading all recipes...")
        load_more_reviews()

        print("Scraping recipe links...")
        recipe_links = scrape_recipe_links()
        print(f"Found {len(recipe_links)} recipes.")

        for idx, recipe_url in enumerate(recipe_links):
            print(f"Scraping recipe {idx + 1}/{len(recipe_links)}: {recipe_url}")
            try:
                recipe_details = extract_recipe_details(recipe_url)
                all_recipes.append(recipe_details)
            except Exception as e:
                print(f"Error scraping recipe {recipe_url}: {e}")
            
            # Save progress periodically
            if (idx + 1) % batch_size == 0:
                batch_count += 1
                filename = f"recipes_batch2_{batch_count}.csv"
                pd.DataFrame(all_recipes).to_csv(filename, index=False)
                print(f"Saved batch {batch_count} to {filename}")
        
        # Save final batch
        pd.DataFrame(all_recipes).to_csv('all_recipes_data.csv', index=False)
        print("All recipes saved to 'all_recipes_data.csv'.")

    except Exception as e:
        print(f"Error during scraping: {e}")

    return all_recipes

# Run the scraper
recipes = scrape_recipes()

# Close the browser
driver.quit()


Loading all recipes...
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' butto

In [3]:
import pandas as pd

In [8]:
df = pd.read_csv('all_recipes_data.csv')
df.head()

,pretitle,title,difficulty,steps
0,Cakes,Sunday Pancakes,Easy,Sift Edmonds Self Raising Flour and Chelsea Wh...
1,Desserts,Banana Cake with Chocolate Icing,Easy,Preheat oven to 180°C conventional / 160° fan ...
2,Desserts,Chocolate Brownie,Easy,Preheat oven to 180°C bake. Line an 18 x 28cm ...
3,Biscuits,Chewy Chocolate Chip Biscuits,Easy,Preheat oven to 180°C bake and grease or line ...
4,Sweets,Basic Buttercream Icing,Easy,"Beat the butter until smooth, pale and fluffy...."


In [11]:
# List of categories to remove
categories_to_remove = ['Savoury', 'No Category', 'Cooking Method - Easy, Fast recipes']

# Filter the DataFrame by excluding these categories
df_filtered = df[~df['pretitle'].isin(categories_to_remove)]

# Display the filtered DataFrame
df_filtered

,pretitle,title,difficulty,steps
0,Cakes,Sunday Pancakes,Easy,Sift Edmonds Self Raising Flour and Chelsea Wh...
1,Desserts,Banana Cake with Chocolate Icing,Easy,Preheat oven to 180°C conventional / 160° fan ...
2,Desserts,Chocolate Brownie,Easy,Preheat oven to 180°C bake. Line an 18 x 28cm ...
3,Biscuits,Chewy Chocolate Chip Biscuits,Easy,Preheat oven to 180°C bake and grease or line ...
4,Sweets,Basic Buttercream Icing,Easy,"Beat the butter until smooth, pale and fluffy...."
...,...,...,...,...
2150,Biscuits,Peanut Butter Chocolate Chip Cookies,Easy,Preheat oven to 180ºC bake and line two large ...
2151,Cakes,Sunday Pancakes,Easy,Sift Edmonds Self Raising Flour and Chelsea Wh...
2152,Desserts,Chocolate Brownie,Easy,Preheat oven to 180°C bake. Line an 18 x 28cm ...
2153,Muffins,Banana Muffins,Easy,Preheat oven to 175°C bake. Line a 12 hole muf...


In [12]:
df_filtered = df.drop(columns=['pretitle'])

In [16]:
# Rename the columns
df = df_filtered.rename(columns={'steps': 'instructions', 'difficulty': 'difficulty_level'})

# Display the DataFrame with the new column names
df

,title,difficulty_level,instructions
0,Sunday Pancakes,Easy,Sift Edmonds Self Raising Flour and Chelsea Wh...
1,Banana Cake with Chocolate Icing,Easy,Preheat oven to 180°C conventional / 160° fan ...
2,Chocolate Brownie,Easy,Preheat oven to 180°C bake. Line an 18 x 28cm ...
3,Chewy Chocolate Chip Biscuits,Easy,Preheat oven to 180°C bake and grease or line ...
4,Basic Buttercream Icing,Easy,"Beat the butter until smooth, pale and fluffy...."
...,...,...,...
2150,Peanut Butter Chocolate Chip Cookies,Easy,Preheat oven to 180ºC bake and line two large ...
2151,Sunday Pancakes,Easy,Sift Edmonds Self Raising Flour and Chelsea Wh...
2152,Chocolate Brownie,Easy,Preheat oven to 180°C bake. Line an 18 x 28cm ...
2153,Banana Muffins,Easy,Preheat oven to 175°C bake. Line a 12 hole muf...


In [17]:
df.to_csv('tia_tsap_scrapedRecipes.csv', index=False)